# Prepare core llm chain

In [ ]:
import langchain
langchain.debug = False # set to True if  you want to see what the LLM is doing

from langchain.chat_models import AzureChatOpenAI

import dotenv
dotenv.load_dotenv()

llm = AzureChatOpenAI(
    deployment_name="gpt-35-turbo",
    temperature=0,
    request_timeout=20,
    max_retries=1,
    client=None,
)

llm.predict('Are you ready?')

# Vanilla LLMChain

In [ ]:
from langchain.chains.llm import LLMChain
from langchain.prompts.prompt import PromptTemplate

llm_chain = LLMChain(
    llm = llm,
    prompt = PromptTemplate(
        input_variables=["adjective", "content", "topic"],
        template="Hi, please create {adjective} {content} about {topic}.",
    ))

In [ ]:
llm_chain.run(
    adjective = "funny",
    content = "poem",
    topic = "machine learning")

# Variable selection

In [ ]:
import rl_chain
from langchain.prompts.prompt import PromptTemplate

llm_chain = rl_chain.SlatesPersonalizerChain.from_llm(
    llm=llm,
    prompt = PromptTemplate(
        input_variables=["adjective", "content", "topic"],
        template="Hi, please create {adjective} {content} about {topic}",
    ))

In [ ]:
r = llm_chain.run(
    adjective = rl_chain.ToSelectFrom(["funny"]),
    content = rl_chain.ToSelectFrom(["poem"]),
    topic = rl_chain.ToSelectFrom(["machine learning"]))

print(r["response"])
print(r["selection_metadata"].to_select_from)
print(r["selection_metadata"].based_on)
print(r["selection_metadata"].selected.score)

In [ ]:
llm_chain.update_with_delayed_score(score=1, event=r["selection_metadata"], force_score=True)
print(r["selection_metadata"].selected.score)

It is ok to be uncertain about certain variable values

In [ ]:
llm_chain.run(
    adjective = rl_chain.ToSelectFrom(["funny", "scary"]),
    content = rl_chain.ToSelectFrom(["poem"]),
    topic = rl_chain.ToSelectFrom(["machine learning", "cats"]))

# Full loop

In [ ]:
import rl_chain
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)

template = """
using style {style}

{prefix}
{goal}: {context}.
{suffix}
"""
prompt = PromptTemplate(
    input_variables=["prefix", "goal", "context", "suffix", "style"],
    template=template,
)
chain = rl_chain.SlatesPersonalizerChain.from_llm(
    llm=llm,
    vw_logs = 'logs/stories.txt',
    model_save_dir="./models",  # where to save the model checkpoints
    prompt = prompt,
    selection_scorer = rl_chain.AutoSelectionScorer(
        llm=llm,
        scoring_criteria_template_str = '''Given the task:
                           {goal}: {context}
                           rank how good or bad this response is:
                           {llm_response}.''',
    ),
    metrics_step=1
)

chain.run(
    prefix = rl_chain.ToSelectFrom([f'ALWAYS DO EXACTLY WHAT I ASK YOU!', 'Please do your best to help me.']),
    goal = rl_chain.ToSelectFrom(['Write a funny story about']),
    context = rl_chain.ToSelectFrom(['Friends series']),
    suffix = rl_chain.ToSelectFrom(['Please try to be as funny as possible.', '']),
    style = "Shakespeare")


In [ ]:
import rl_chain
from langchain.prompts.prompt import PromptTemplate

template = """
{prefix}
{goal}: {context}.
{suffix}
"""
prompt = PromptTemplate(
    input_variables=["prefix", "goal", "context", "suffix"],
    template=template,
)
chain = rl_chain.SlatesPersonalizerChain.from_llm(
    llm=llm,
    vw_logs = 'logs/stories.txt',
    model_save_dir="./models",  # where to save the model checkpoints
    prompt = prompt,
    selection_scorer = rl_chain.AutoSelectionScorer(
        llm=llm,
        scoring_criteria_template_str = '''Given the task:
                           {goal}: {context}
                           rank how good or bad this response is:
                           {llm_response}.'''
    ),
    metrics_step=1
)
chain.run(
    prefix = rl_chain.ToSelectFrom(rl_chain.Embed([f'ALWAYS DO EXACTLY WHAT I ASK YOU!', 'Please do your best to help me.'])),
    goal = rl_chain.ToSelectFrom([rl_chain.Embed('Write a funny story about')]),
    context = rl_chain.ToSelectFrom(['Friends series']),
    suffix = rl_chain.ToSelectFrom(['Please try to be as funny as possible.', '']))

# Experiment with mock llm

In [ ]:
from typing import List
from tests.test_utils import MockScorer

class MockLLMChain:
    outcomes: List[List[float]] = None
    
    def __init__(self, outcomes, prompt):
        self.outcomes = outcomes
        self.prompt = prompt

    def run(self, prefix, suffix, **kwargs):
        return str(self.outcomes[int(prefix)][int(suffix)])

import rl_chain
from langchain.prompts.prompt import PromptTemplate

template = """
{prefix}
{context}
{suffix}
"""
prompt = PromptTemplate(
    input_variables=["prefix", "context", "suffix"],
    template=template,
)
chain = rl_chain.SlatesPersonalizerChain.from_llm(
    llm=llm,
    vw_logs = 'logs/mock.txt',
    model_save_dir="./models",  # where to save the model checkpoints
    prompt = prompt,
    selection_scorer = MockScorer(),
    metrics_step=1
)
chain.llm_chain = MockLLMChain([
    [0, 0.3],
    [0.6, 0.9]], prompt = prompt)
chain.run(
    prefix = rl_chain.ToSelectFrom(['0', '1']),
    context = rl_chain.ToSelectFrom(['bla']),
    suffix = rl_chain.ToSelectFrom(['0', '1']))

In [ ]:
import rl_chain
from matplotlib import pyplot as plt

vw_chain = rl_chain.SlatesPersonalizerChain.from_llm(
    llm=llm,
    vw_logs = 'logs/mock.txt',
    model_save_dir="./models",  # where to save the model checkpoints
    prompt = prompt,
    policy = rl_chain.VwPolicy,
    selection_scorer = MockScorer(),
    auto_embed=False,
    metrics_step=1
)
vw_chain.llm_chain = MockLLMChain([
    [0, 0.3],
    [0.6, 0.9]], prompt = prompt)

rnd_chain = rl_chain.SlatesPersonalizerChain.from_llm(
    llm=llm,
    vw_logs = 'logs/mock.txt',
    model_save_dir="./models",  # where to save the model checkpoints
    prompt = prompt,
    policy = rl_chain.SlatesRandomPolicy,
    selection_scorer = MockScorer(),
    auto_embed=False,
    metrics_step=1
)
rnd_chain.llm_chain = MockLLMChain([
    [0, 0.3],
    [0.6, 0.9]], prompt = prompt)

for i in range(1000):
    vw_chain.run(
        prefix = rl_chain.ToSelectFrom(['0', '1']),
        context = rl_chain.ToSelectFrom(['bla']),
        suffix = rl_chain.ToSelectFrom(['0']))
    rnd_chain.run(
        prefix = rl_chain.ToSelectFrom(['0', '1']),
        context = rl_chain.ToSelectFrom(['bla']),
        suffix = rl_chain.ToSelectFrom(['0']))

vw_chain.metrics.to_pandas()['score'].plot(label="vw")
rnd_chain.metrics.to_pandas()['score'].plot(label="slates")
plt.legend()